In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
from obspy.io.segy.segy import _read_segy
import ipywidgets as widgets
from IPython.display import display, clear_output



# Folder with SEG-Y files
data_folder = "data"
sgy_files = sorted([f for f in os.listdir(data_folder) if f.endswith(".sgy")])

# Dropdown widget
dropdown = widgets.Dropdown(
    options=sgy_files,
    description='Select file:',
    layout=widgets.Layout(width='50%')
)

# Output widget for plot
out = widgets.Output()

# Callback function
def on_file_selected(change):
    out.clear_output(wait=True)
    file = change['new']
    file_path = os.path.join(data_folder, file)

    with out:
        print(f"\nReading: {file_path}")
        try:
            st = _read_segy(file_path, headonly=False)
            print(f"  Traces: {len(st.traces)} | Samples per trace: {len(st.traces[0].data)}")
            data = np.array([tr.data for tr in st.traces])

            # Plot
            plt.figure(figsize=(10, 6))
            plt.imshow(data.T, cmap="seismic", aspect="auto")
            plt.title(f"DAS Data: {file}")
            plt.xlabel("Trace Number")
            plt.ylabel("Time Samples")
            plt.colorbar(label="Amplitude")
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"Error reading {file}: {e}")

# Trigger plot when file is selected
dropdown.observe(on_file_selected, names='value')

# Display
display(dropdown, out)

# Force first plot to render
on_file_selected({'new': dropdown.value})


Dropdown(description='Select file:', layout=Layout(width='50%'), options=('FORGE_78-32_iDASv3-P11_UTC190424172…

Output()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from obspy.io.segy.segy import _read_segy
import ipywidgets as widgets
from IPython.display import display, clear_output

%matplotlib inline

# === Load SEG-Y file list ===
data_folder = "data"
sgy_files = sorted([f for f in os.listdir(data_folder) if f.endswith(".sgy")])

print(f"Found {len(sgy_files)} SEG-Y files.")

# === Global storage to cache current data ===
current_data = {}

# === Widgets ===
dropdown = widgets.Dropdown(
    options=sgy_files,
    description='Select file:',
    layout=widgets.Layout(width='50%')
)

slider = widgets.IntRangeSlider(
    value=[0, 200],
    min=0,
    max=200,  # Temporary default, will update after file is loaded
    step=1,
    description='Time range:',
    continuous_update=False,
    layout=widgets.Layout(width='80%')
)

out = widgets.Output()

# === Update Plot ===
def update_plot(file, tmin, tmax):
    out.clear_output(wait=True)

    with out:
        try:
            # Read or reuse
            if file not in current_data:
                st = _read_segy(os.path.join(data_folder, file), headonly=False)
                data = np.array([tr.data for tr in st.traces])
                current_data[file] = data
            else:
                data = current_data[file]

            tmax = min(tmax, data.shape[1])

            # Plot
            plt.figure(figsize=(10, 6))
            plt.imshow(data[:, tmin:tmax].T, cmap="seismic", aspect="auto", origin='upper')
            plt.title(f"{file}  |  Time samples: {tmin} to {tmax}")
            plt.xlabel("Trace Number")
            plt.ylabel("Time Samples")
            plt.colorbar(label="Amplitude")
            plt.tight_layout()
            plt.show()

        except Exception as e:
            print(f"Error: {e}")

# === Callback for file selection ===
def on_file_selected(change):
    file = change['new']
    try:
        st = _read_segy(os.path.join(data_folder, file), headonly=True)
        sample_len = len(st.traces[0].data)
        slider.max = sample_len
        slider.value = [0, sample_len]
    except Exception as e:
        print(f"Could not load file: {e}")
    update_plot(file, *slider.value)

# === Callback for time slider ===
def on_slider_change(change):
    update_plot(dropdown.value, *change['new'])

# === Register Callbacks ===
dropdown.observe(on_file_selected, names='value')
slider.observe(on_slider_change, names='value')

# === Display ===
display(dropdown, slider, out)

# === Initial trigger ===
on_file_selected({'new': dropdown.value})


Found 19 SEG-Y files.


Dropdown(description='Select file:', layout=Layout(width='50%'), options=('FORGE_78-32_iDASv3-P11_UTC190424172…

IntRangeSlider(value=(0, 200), continuous_update=False, description='Time range:', layout=Layout(width='80%'),…

Output()